Cluster the orange ball using Single Guassian

In [1]:
# import packages
import cv2
import os
import numpy as np
import math
import scipy
# from datetime import datetime # for benchmarking purpose

In [2]:
def cal_mean_cov(img):
    l, w, h = img.shape
    mean = [[np.sum(img[:,:,0])/(l*w)],[np.sum(img[:,:,1])/(l*w)],[np.sum(img[:,:,2])/(l*w)]]
    cov = np.zeros((3,3),)
    #R_value = []
    for width in range(len(img[:,0,0])):
        for length in range(len(img[0,:,0])):
            RGB_value = [[img[width][length][0]],[img[width][length][1]],[img[width][length][2]]]
            cov = cov + (np.asmatrix(RGB_value) - np.asmatrix(mean))@(np.asmatrix(RGB_value) - np.asmatrix(mean)).T
            #R_value.append(img[width][length][0])
    cov = cov/(l*w)
    #print("size of image: ", l*w)
    #print("variance for R is: ", np.var(R_value))
    return mean,cov

In [3]:
def cal_mean_cov_vectorized(X):
    N, D = X.shape
    mean = X.mean(axis = 0) # compute mean
    cov = np.matmul((X - mean).T, (X - mean))/(N-1) # compute covariance
    return mean, cov

In [4]:
# load data
input_dir = "train_images"# path to the train image dataset
# output directory
output_dir = "single_gaussian_result"
# User defined threshold
tau = 0.00000000000000001
prior = 0.5

for img_name in os.listdir(input_dir):
    img = cv2.imread(os.path.join(input_dir, img_name))
    l, w, h = img.shape # original shape of 2D image
    X = img.transpose(2,0,1).reshape(3,-1).T # reshape to num of rows = num of pixels, num of column = 3 (RGB)
    N, D = X.shape
    #img = X.reshape(l, w, -1) # reshape back to 2d image

    """
    # element-wise algorithm (original)
    print("start element-wise algorithm", datetime.now().strftime("%H:%M:%S"))
    mean, cov = cal_mean_cov(img)

    # creat a mask to indicate the position of ball
    mask = np.zeros((img.shape[:-1]))
    #print("cov: \n", cov)
    
    likelihood_list = []
    
    for width in range(len(img[:,0,0])):
        for length in range(len(img[0,:,0])):
            curr_pixel = np.asmatrix([[img[width][length][0]],[img[width][length][1]],[img[width][length][2]]])
            likelihood = 1/(math.sqrt(((2*math.pi)**3)* np.linalg.det(cov))) * math.exp((-0.5)*(curr_pixel-mean).T @ np.linalg.inv(cov)@ (curr_pixel-mean))
            #print("likelihood", likelihood)
            likelihood_list.append(likelihood)
            if (likelihood * prior < tau):
                mask[width][length] = 1
    #print(likelihood_list)
    if (likelihood * prior < tau):
        mask[width][length] = 1

    print("end element-wise algorithm, start vectorized algorithm", datetime.now().strftime("%H:%M:%S"))
    """
    
    # vectorized algorithm
    mean, cov = cal_mean_cov_vectorized(X)    
    
    # calculate likelihood using gaussian distribution
    # each pixel is row of X
    constant_in_likelihood = 1/(math.sqrt(((2*math.pi)**3)* np.linalg.det(cov))) 
    sigma_inv = np.linalg.inv(cov)
    X2 = X-mean
    exponent = (-0.5)*(np.dot(X2, sigma_inv) * X2).sum(1) 
    likelihood = constant_in_likelihood * np.exp(exponent)
    
    # posterior
    prior = 0.5
    posterior = prior* likelihood 
    
    # mask
    mask = posterior
    mask[mask < tau] = -1 # temperarily mark as -1 to prevent being marked as 0 later
    mask[mask != -1] = 0
    mask[mask == -1] = 1
    mask = mask.reshape(l, w) # reshape back to 2D image
    # print("end vectorized algorithm", datetime.now().strftime("%H:%M:%S"))

    ##  show mask    
    three_d_mask = np.stack((mask,mask,mask),axis=2)
    masked_img = np.multiply(three_d_mask, img)
    image_name = os.path.join(output_dir,"masked_"+ str(img_name))
    cv2.imshow(image_name, masked_img)
    cv2.waitKey(0)
    print("Finish Generating mask for image ", str(img_name))
print("Complete Process All Images")




Finish Generating mask for image  99.jpg
Finish Generating mask for image  248.jpg
Finish Generating mask for image  208.jpg
Finish Generating mask for image  144.jpg
Finish Generating mask for image  256.jpg
Finish Generating mask for image  223.jpg
Finish Generating mask for image  68.jpg
Finish Generating mask for image  91.jpg
Finish Generating mask for image  231.jpg
Finish Generating mask for image  76.jpg
Finish Generating mask for image  168.jpg
Finish Generating mask for image  152.jpg
Finish Generating mask for image  216.jpg
Finish Generating mask for image  106.jpg
Finish Generating mask for image  264.jpg
Finish Generating mask for image  192.jpg
Finish Generating mask for image  114.jpg
Finish Generating mask for image  200.jpg
Finish Generating mask for image  121.jpg
Finish Generating mask for image  137.jpg
Finish Generating mask for image  160.jpg
Finish Generating mask for image  176.jpg
Finish Generating mask for image  280.jpg
Complete Process All Images


In [5]:
v = "single_gaussian_result\masked_train_images\106.jpg"
v.split('\\')

['single_gaussian_result', 'masked_train_imagesF.jpg']

In [6]:
matrixSize = 3
A = scipy.random.rand(matrixSize,matrixSize)
B = np.dot(A,A.transpose())
print ('random positive semi-define matrix for today is', B)


random positive semi-define matrix for today is [[1.69268442 0.34390751 1.05759914]
 [0.34390751 0.11531918 0.22644026]
 [1.05759914 0.22644026 0.72508576]]
